In [98]:
import pandas as pd

df = pd.read_excel("test.xlsx")
df.head()

,menu_name,ingredients,characteristics,menu_category
0,Blossom Coconut Roll,"แป้งเค้ก, มะพร้าวขูด, น้ำตาล, ไข่, เนย","หวาน, มัน","ของหวาน, เบเกอรี/เค้ก"
1,Lychee Rose Juice Mixed with Pomegranate,"ลิ้นจี่, น้ำกุหลาบ, น้ำทับทิม, น้ำตาล, น้ำแข็ง","หวาน, เปรี้ยว, มัน",เครื่องดื่ม/น้ำผลไม้
2,Mango Yogurt Cake,"มะม่วง, โยเกิร์ต, แป้งเค้ก, น้ำตาล, ไข่, เนย","หวาน, มัน","ของหวาน, เบเกอรี/เค้ก"
3,Honey Nougat Cream Frappuccino,"นม, น้ำผึ้ง, นูกัต, ครีม, กาแฟ, น้ำแข็ง","หวาน, มัน",เครื่องดื่ม/น้ำผลไม้
4,Honey Nougat Oatmilk Macchiato,"นมข้าวโอ๊ต, น้ำผึ้ง, นูกัต, กาแฟ","หวาน, มัน",เครื่องดื่ม/น้ำผลไม้


In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer

df_sheet2 = df.copy()

# Combine ingredients and characteristics into a single text field for embeddings
df_sheet2["combined_features"] = df_sheet2["ingredients"] + ", " + df_sheet2["characteristics"] + ", " + df_sheet2["menu_category"]

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(", "))

# Fit and transform the text data into embeddings
tfidf_matrix = vectorizer.fit_transform(df_sheet2["combined_features"])

# Convert the matrix to a DataFrame for better readability
feature_names = vectorizer.get_feature_names_out()
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

C:\Users\User\AppData\Roaming\Python\Python39\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [100]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Convert to DataFrame for better readability
df_cosine_sim = pd.DataFrame(cosine_sim_matrix, index=df_sheet2["menu_name"], columns=df_sheet2["menu_name"])


In [101]:
# Function to recommend similar menus based on liked menu
def get_recommendations(liked_menu, df_cosine_sim, top_n=5):
    if liked_menu not in df_cosine_sim.index:
        return f"Menu '{liked_menu}' not found in dataset."
    
    # Get similarity scores for the liked menu
    sim_scores = df_cosine_sim[liked_menu].sort_values(ascending=False)
    
    # Exclude the liked menu itself and get top N recommendations
    recommended_menus = sim_scores.iloc[1:top_n+1].index.tolist()
    
    return recommended_menus

# Display function to allow user input
def recommend_based_on_input(liked_menu):
    recommendations = get_recommendations(liked_menu, df_cosine_sim)
    return recommendations

# Display menu names for user reference
menu_list = df_sheet2["menu_name"].tolist()


In [102]:
recommend_based_on_input("Blossom Coconut Roll")

#Show the ingredients and characteristics of selected menu
df_sheet2[df_sheet2["menu_name"] == "Blossom Coconut Roll"][["menu_name", "ingredients", "characteristics", "menu_category"]]

,menu_name,ingredients,characteristics,menu_category
0,Blossom Coconut Roll,"แป้งเค้ก, มะพร้าวขูด, น้ำตาล, ไข่, เนย","หวาน, มัน","ของหวาน, เบเกอรี/เค้ก"


In [103]:
#Show the ingredients and characteristics of the recommended menu
df_sheet2[df_sheet2["menu_name"].isin(recommend_based_on_input("Blossom Coconut Roll"))][["menu_name", "ingredients", "characteristics", "menu_category"]]

,menu_name,ingredients,characteristics,menu_category
2,Mango Yogurt Cake,"มะม่วง, โยเกิร์ต, แป้งเค้ก, น้ำตาล, ไข่, เนย","หวาน, มัน","ของหวาน, เบเกอรี/เค้ก"
102,Glazed Donut,"แป้ง, น้ำตาล, ไข่, นม","หวาน, มัน","ของหวาน, เบเกอรี/เค้ก"
104,Chocolate Glazed Donut,"แป้ง, ช็อกโกแลต, น้ำตาล, ไข่, นม","หวาน, ขม, มัน","ของหวาน, เบเกอรี/เค้ก"
303,เฮอร์ชี่ส์ ช็อกโกแลต ชิพ คุกกี้,"แป้ง, ช็อกโกแลตชิพ, น้ำตาล, เนย","หวาน, มัน, หอมช็อกโกแลต","ของหวาน, เบเกอรี/เค้ก"
331,เซตปังปิ้งเนย,"ขนมปัง, เนย","หวาน, มัน, กรอบ","ของหวาน, เบเกอรี/เค้ก"


In [104]:
liked_menus = ["ฮัทปีกไก่แซ่บ", "หอมทอด", "ชาไทยนมสด"]
menu_names = menu_list
liked_indices = [i for i, name in enumerate(menu_names) if name in liked_menus]


In [105]:
import numpy as np

# Get the vectors for liked menus
liked_vectors = tfidf_matrix[liked_indices]

# Average the vectors to create a user profile vector
user_vector = liked_vectors.mean(axis=0)


In [106]:
from sklearn.metrics.pairwise import cosine_similarity
user_vector_array = np.asarray(user_vector)
similarity_scores = cosine_similarity(user_vector_array, tfidf_matrix).flatten()
scored_menus = [
    (menu_names[i], similarity_scores[i])
    for i in np.argsort(similarity_scores)[::-1]
    if menu_names[i] not in liked_menus
]

top_n = 30
top_recommendations = scored_menus[:top_n]


In [107]:
# Print or return
for menu, score in top_recommendations:
    print(f"{menu} — Score: {score:.4f}")

ฮัทปีกไก่ทอด (สูตรต้นตำรับ) — Score: 0.5998
เคอร์ลี่ ฟรายส์ — Score: 0.5219
กาแฟ - ชาไทย — Score: 0.4758
เฟรนช์ฟรายส์ — Score: 0.4746
โกโก้ - ชาไทย — Score: 0.4574
ปังเย็นชาไทย — Score: 0.4025
ไก่ทอดเกาหลี ซอสการ์ลิค — Score: 0.3964
ชิกเก้นป๊อป — Score: 0.3857
ฟิช แอนด์ ชิปส์ — Score: 0.3759
นิวออร์ลีนส์สไปซี่ — Score: 0.2919
ชาไทย คาราเมล — Score: 0.2838
หมึกทอดกรอบ — Score: 0.2834
พิซซ่าเครซี่ เบคอน — Score: 0.2823
พิซซ่า เครซี่ เบคอน — Score: 0.2823
สปาเก็ตตี้อิตาเลียนชิกเก้น — Score: 0.2796
ฟิชโดนัท พร้อมซอสบาร์บีคิว — Score: 0.2521
ลาเต้ — Score: 0.2389
ไก่นิวออร์ลีนส์ — Score: 0.2377
น้ำพริกเผา — Score: 0.2359
เซตปังปิ้งเนย — Score: 0.2343
ชีสซี่การ์ลิคสติ๊ก — Score: 0.2298
ฟิชโนริไบท์ — Score: 0.2294
Coffee Frappuccino — Score: 0.2211
Espresso Frappuccino — Score: 0.2211
ปีกกลางทอดน้ำปลา — Score: 0.2191
Glazed Donut — Score: 0.2142
Everything Bagel — Score: 0.2136
โกโก้นมสด — Score: 0.2099
กุ้งคั่วพริกเกลือราดข้าว — Score: 0.2087
Caramel Cream Frappuccino — Score: 0.2071


In [108]:
# Display liked menus with their ingredients and characteristics and menu category
df_sheet2[df_sheet2["menu_name"].isin(liked_menus)][["menu_name", "ingredients", "characteristics", "menu_category"]]

,menu_name,ingredients,characteristics,menu_category
289,ฮัทปีกไก่แซ่บ,"ไก่ทอด, พริก, เครื่องเทศ","เผ็ด, เค็ม, กรอบ","ฟาสต์ฟู้ด/จานด่วน, อาหารอเมริกัน"
294,หอมทอด,"หัวหอม, แป้ง, เกลือ","เค็ม, มัน, กรอบ, หอมหวาน",ฟาสต์ฟู้ด/จานด่วน
305,ชาไทยนมสด,"ชาไทย, นม, น้ำตาล","หวาน, มัน, หอมชาไทย",เครื่องดื่ม/น้ำผลไม้


,menu_name,ingredients,characteristics,menu_category
60,Espresso Frappuccino,"นม, น้ำแข็ง, กาแฟ, น้ำตาล","หวาน, ขม, มัน",เครื่องดื่ม/น้ำผลไม้
61,Coffee Frappuccino,"นม, น้ำแข็ง, กาแฟ, น้ำตาล","หวาน, ขม, มัน",เครื่องดื่ม/น้ำผลไม้
65,Caramel Cream Frappuccino,"นม, น้ำแข็ง, คาราเมล, น้ำตาล","หวาน, มัน",เครื่องดื่ม/น้ำผลไม้
102,Glazed Donut,"แป้ง, น้ำตาล, ไข่, นม","หวาน, มัน","ของหวาน, เบเกอรี/เค้ก"
133,น้ำพริกเผา,"พริก, กระเทียม, น้ำตาล, น้ำปลา","เผ็ด, หวาน, เค็ม",อาหารไทย
155,ปีกกลางทอดน้ำปลา,"ปีกไก่, น้ำปลา, น้ำมัน","เค็ม, มัน, กรอบ",อาหารไทย
190,กุ้งคั่วพริกเกลือราดข้าว,"กุ้ง, พริก, เกลือ, ข้าว","เผ็ด, เค็ม, หอม","อาหารจานเดียว, อาหารทะเล, อาหารไทย"
234,พิซซ่าเครซี่ เบคอน,"แป้งพิซซ่า, เบคอน, ชีส, ซอสมะเขือเทศ","เค็ม, มัน, กรอบ","พิซซ่า, อาหารอเมริกัน, อาหารอิตาเลียน"
256,พิซซ่า เครซี่ เบคอน,"แป้งพิซซ่า, เบคอน, ชีส, ซอสมะเขือเทศ","เค็ม, มัน, กรอบ","พิซซ่า, อาหารอเมริกัน, อาหารอิตาเลียน"
270,ฟิช แอนด์ ชิปส์,"ปลาชุบแป้งทอด, มันฝรั่งทอด, เกลือ","เค็ม, มัน, กรอบ","อาหารอเมริกัน, ฟาสต์ฟู้ด/จานด่วน, อาหารทะเล"


In [109]:
# Display the recommended menus with their ingredients and characteristics and menu category
df_sheet2[df_sheet2["menu_name"].isin(recommendations)][["menu_name", "ingredients", "characteristics", "menu_category"]]

,menu_name,ingredients,characteristics,menu_category
60,Espresso Frappuccino,"นม, น้ำแข็ง, กาแฟ, น้ำตาล","หวาน, ขม, มัน",เครื่องดื่ม/น้ำผลไม้
61,Coffee Frappuccino,"นม, น้ำแข็ง, กาแฟ, น้ำตาล","หวาน, ขม, มัน",เครื่องดื่ม/น้ำผลไม้
65,Caramel Cream Frappuccino,"นม, น้ำแข็ง, คาราเมล, น้ำตาล","หวาน, มัน",เครื่องดื่ม/น้ำผลไม้
102,Glazed Donut,"แป้ง, น้ำตาล, ไข่, นม","หวาน, มัน","ของหวาน, เบเกอรี/เค้ก"
133,น้ำพริกเผา,"พริก, กระเทียม, น้ำตาล, น้ำปลา","เผ็ด, หวาน, เค็ม",อาหารไทย
155,ปีกกลางทอดน้ำปลา,"ปีกไก่, น้ำปลา, น้ำมัน","เค็ม, มัน, กรอบ",อาหารไทย
190,กุ้งคั่วพริกเกลือราดข้าว,"กุ้ง, พริก, เกลือ, ข้าว","เผ็ด, เค็ม, หอม","อาหารจานเดียว, อาหารทะเล, อาหารไทย"
234,พิซซ่าเครซี่ เบคอน,"แป้งพิซซ่า, เบคอน, ชีส, ซอสมะเขือเทศ","เค็ม, มัน, กรอบ","พิซซ่า, อาหารอเมริกัน, อาหารอิตาเลียน"
256,พิซซ่า เครซี่ เบคอน,"แป้งพิซซ่า, เบคอน, ชีส, ซอสมะเขือเทศ","เค็ม, มัน, กรอบ","พิซซ่า, อาหารอเมริกัน, อาหารอิตาเลียน"
270,ฟิช แอนด์ ชิปส์,"ปลาชุบแป้งทอด, มันฝรั่งทอด, เกลือ","เค็ม, มัน, กรอบ","อาหารอเมริกัน, ฟาสต์ฟู้ด/จานด่วน, อาหารทะเล"
